In [1]:
# ----- imports
import numpy as np
import matplotlib.pyplot as plt
import clipboard
from scipy.constants import c
import pynlo
from ipywidgets import interact, widgets, fixed
from tqdm import tqdm
from numpy.fft import fftshift, ifftshift


NOT USING MKL FOR FFT'S IN PYNLO


In [2]:
fft = lambda x, axis=-1, fsc=1.0: fftshift(np.fft.fft(ifftshift(x, axes=axis), axis=axis), axes=axis) * fsc
ifft = lambda x, axis=-1, fsc=1.0: fftshift(np.fft.ifft(ifftshift(x, axes=axis), axis=axis), axes=axis) / fsc
rfft = lambda x, axis=-1, fsc=1.0: np.fft.rfft(ifftshift(x, axes=axis), axis=axis) * fsc
irfft = lambda x, axis=-1, fsc=1.0: fftshift(np.fft.irfft(x, axis=axis), axes=axis) / fsc

In [3]:
loss_ins = 10 ** (-0.7 / 10)
loss_spl = 10 ** (-0.7 / 10)
loss_mat = 10 ** (-1 / 10)

f_r = 200e6
n = 256
v_min = c / 1750e-9
v_max = c / 1400e-9
v0 = c / 1560e-9
e_p = 1e-9  # doesn't matter, will do a direct replacement of a_t later

t_fwhm = 2e-12
min_time_window = 20e-12
pulse = pynlo.light.Pulse.Sech(
    n,
    v_min,
    v_max,
    v0,
    e_p,
    t_fwhm,
    min_time_window,
    alias=2,
)
dv_dl = pulse.v_grid**2 / c  # J / Hz -> J / m

changing n from 256 to 864 to support both time and frequency bandwidths


In [4]:
path = r"../sim_output/200MHz_osc_v2/post_chirp_sweep/"

P_V = np.load(path + "P_V.npy", mmap_mode="r")
P_WL = P_V * dv_dl
P_T = np.load(path + "P_T.npy", mmap_mode="r")
V_W = np.load(path + "V_W.npy", mmap_mode="r")
T_W = np.load(path + "T_W.npy", mmap_mode="r")
E_P = np.sum(P_V * pulse.dv, axis=-1)


In [5]:
A_V = np.load(path + "A_V.npy", mmap_mode="r")

In [6]:
spec = np.genfromtxt("../Sichong/osc_build_v2/edfa_fb.CSV", skip_header=44, delimiter=",")
spec[:, 1] = 10 ** (spec[:, 1] / 10)  # dB -> linear
spec[:, 0] = c / (spec[:, 0] * 1e-9)  # wavelength -> frequency
spec[:, 1] *= c / spec[:, 0] ** 2  # J / m -> J / Hz
spec[:, 1] /= spec[:, 1].max()  # normalize
p_data = pulse.copy()
p_data.import_p_v(spec[:, 0], spec[:, 1], phi_v=np.zeros(spec[:, 1].size))

In [7]:
corr = np.genfromtxt("../Sichong/02-07-2024_FROG/edfa_fb_pump.txt")
t = corr[:, 0][1:] * 1e-15
nu = c / (corr[0][1:] * 1e-9)
frog = corr[:, 1:][1:]
frog *= c / nu ** 2
frog /= frog.max()

# fig, ax = plt.subplots(1, 1)
# ax.pcolormesh(t*1e15, nu*1e-12 / 2, frog.T)

In [8]:
from scipy.integrate import simpson
from scipy.interpolate import InterpolatedUnivariateSpline
frog -= frog[0]
corr = simpson(frog[:, ::-1], axis=1, x=nu[::-1])
corr /= corr.max()

corr = InterpolatedUnivariateSpline(t, corr, ext=1)(pulse.t_grid)

In [21]:
fig, axs = plt.subplots(2, 3, figsize=np.array([9.76, 6.39]))
ax1, ax2, ax3, ax4 = axs[:, :-1].flatten()
# ax1_ = ax1.twinx()
# ax1_.set_ylabel("instantaneous time (fs)")
ax5, ax6 = axs[:, -1].flatten()
ax5_ = ax5.twinx()
ax5_.set_ylabel("instantaneous frequency (THz)")
ax1.semilogy(p_data.wl_grid * 1e9, p_data.p_v * dv_dl, "k--", linewidth=2)
# ax1.set_ylim(-P_WL.max() * .01, P_WL.max() * 1.01)
ax1.set_ylim(1e-5, P_WL.max())
ax2.plot(pulse.t_grid * 1e15, corr, "k--", linewidth=2)
ax2.set_ylim(-.05, 1.05)
ax5.set_ylim(-.05, 1.05)
img_v_w = ax3.pcolormesh(V_W * 1e-12)
img_t_w = ax4.pcolormesh(T_W * 1e15)
v_grid = pulse.v_grid * 1e-12
wl_grid = pulse.wl_grid
img = ax6.pcolorfast(pulse.t_grid * 1e15, v_grid, np.ones((pulse.n - 1, pulse.n - 1)), vmin=0, vmax=1)
ax6.set_xlabel("time (fs)")
ax6.set_ylabel("frequency (THz)")
# img = ax6.imshow(np.ones((pulse.n, pulse.n)), vmin=0, vmax=1)
plt.colorbar(img_v_w)
plt.colorbar(img_t_w)

ax1.set_xlabel("wavelength (nm)")
ax2.set_xlabel("time (fs)")
ax2.set_title("Intensity autocorrelation")
ax3.set_xlabel("post-chirp (mm)")
ax3.set_ylabel("pre-chirp (mm)")
ax4.set_xlabel("post-chirp (mm)")
ax4.set_ylabel("pre-chirp (mm)")
ax3.set_title("bandwidth (THz)")
ax4.set_title("pulse duration (fs)")
fig.tight_layout()

initialized = False

In [22]:
@interact(
    idx_pre=widgets.IntSlider(min=0,max=P_WL.shape[0] - 1,step=1),
    idx_post=widgets.IntSlider(min=0,max=P_WL.shape[1] - 1,step=1),
)
def update(idx_pre, idx_post):
    assert isinstance(idx_pre, int)
    assert isinstance(idx_post, int)
    
    global img1
    global img2
    global initialized
    global l1
    global l2
    global l3
    global p1
    global p2
    global img
    global l1_
    global l2_
    
    p_wl = P_WL[idx_pre, idx_post]
    p_t = P_T[idx_pre, idx_post]
    a_t = ifft(A_V[idx_pre, idx_post], fsc=pulse.dt)
    signal = np.c_[a_t] * np.ones(pulse.n)
    gate = np.c_[np.ones(pulse.n)] * a_t
    for r in range(pulse.n):
        gate[r] = np.roll(gate[r], -r)
    gate = fftshift(gate, axes=1)
    frog = abs(fft(signal * gate, axis=0, fsc=pulse.dt)) ** 2
    icorr = np.sum(frog, axis=0) * pulse.dv
    
    pulse.a_t[:] = a_t
    idx_v = ((pulse.p_v / pulse.p_v.max()) > 1e-2).nonzero()
    idx_t = ((pulse.p_t / pulse.p_t.max()) > 1e-2).nonzero()
    if not initialized:
        l1, = ax1.semilogy(pulse.wl_grid * 1e9, p_wl)
        # l1_, = ax1_.plot(pulse.wl_grid[idx_v] * 1e9, pulse.tg_v[idx_v] * 1e15, "C1")
        l2, = ax2.plot(pulse.t_grid * 1e15, icorr / icorr.max())
        # l2_, = ax5_.plot(pulse.t_grid[idx_t] * 1e15, pulse.vg_t[idx_t] * 1e-12, "C1")
        l3, = ax5.plot(pulse.t_grid * 1e15, p_t / p_t.max())
        p1, = ax3.plot(idx_post, idx_pre, "ko")
        p2, = ax4.plot(idx_post, idx_pre, "ko")
        # img.set_data(pulse.t_grid * 1e15, wl_grid, A=(frog/frog.max())[:-1, :-1])
        img.set_data(A=(frog/frog.max())[:-1, :-1])
        initialized=True
        fig.tight_layout()
    else:
        l1.set_ydata(p_wl)
        l2.set_ydata(icorr / icorr.max())
        l3.set_ydata(p_t / p_t.max())
        # l1_.set_xdata(pulse.wl_grid[idx_v] * 1e9)
        # l2_.set_xdata(pulse.t_grid[idx_t] * 1e15)
        # l1_.set_ydata(pulse.tg_v[idx_v] * 1e15)
        # l2_.set_ydata(pulse.vg_t[idx_t] * 1e-12)
        p1.set_xdata(np.array([idx_post]))
        p1.set_ydata(np.array([idx_pre]))
        p2.set_xdata(np.array([idx_post]))
        p2.set_ydata(np.array([idx_pre]))
        # img.set_data(pulse.t_grid * 1e15, wl_grid, A=(frog/frog.max())[:-1, :-1])
        img.set_data(A=(frog/frog.max())[:-1, :-1])
        

interactive(children=(IntSlider(value=0, description='idx_pre', max=299), IntSlider(value=0, description='idx_…

In [11]:
E_P.mean() * f_r * 1e3

331.83543130848255

In [12]:
T_W.min() * 1e15

68.35914485538258

In [13]:
np.unravel_index(T_W.argmin(), T_W.shape)

(53, 85)

In [14]:
print(E_P.max() * f_r, "max power")
print(E_P.min() * f_r, "min power")

0.34708398485114883 max power
0.3076920540373737 min power


In [15]:
# plt.figure()
# plt.pcolormesh(V_W/T_W)

In [16]:
# fig, ax = plt.subplots(1, 1)
# ax.semilogy(p_data.wl_grid*1e6, p_data.p_v * p_data.v_grid**2/c)
# ax.set_xlabel("wavelength (nm)")
# ax.set_ylabel("a.u.")

/Users/peterchang/miniconda3/envs/idp/lib/python3.8/site-packages/ipykernel/eventloops.py:128: UserWarning: Attempt to set non-positive ylim on a log-scaled axis will be ignored.
  el.exec() if hasattr(el, 'exec') else el.exec_()
